In [1]:
# importing libraries 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
import json
from pydub.utils import make_chunks
from scipy.spatial.distance import pdist, squareform, euclidean, cosine
from spectralcluster import SpectralClusterer
import moviepy.editor as mp
# filename = "file.wav"
from resemblyzer import sampling_rate


In [2]:
# video = mp.VideoFileClip("test2.mp4")
# video.audio.write_audiofile(video.filename.split('.')[0]+".wav")
# filename = video.filename.split('.')[0]+'.wav'
# wav_fpath = Path(filename)

In [3]:
filename = "file.wav"
wav_fpath = Path(filename)

In [4]:
# video.filename.split('.')[0]+'.wav'

In [5]:


clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    #p_percentile=0.90,
    #gaussian_blur_sigma=1
    )

encoder = VoiceEncoder("cpu")

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunk_length_ms = 120000 # pydub calculates in millisec
    chunks = make_chunks(sound, chunk_length_ms) #M
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_json = ""
    labelling = []
    global_label = []
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        print('start preprocessing:' + chunk_filename)
        # recognize the chunk
        #wav = preprocess_wav(wav_fpath)
        wav = preprocess_wav(chunk_filename)
        print('preprocessing done:' + chunk_filename)
        #wav = audio_chunk
        _, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
        labels = clusterer.predict(cont_embeds)
        times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
        
        start_time = 0
        print('create labels:' + chunk_filename)
#         print(times)
        perChunkLabel = []
        for j,time in enumerate(times):
            if j>0 and labels[j]!=labels[j-1]:
                temp = [str(labels[j-1]),start_time,time]
                perChunkLabel.append(temp)
                start_time = time
            if j==len(times)-1:
                temp = [str(labels[j]),start_time,time]
                perChunkLabel.append(temp)
#         chunkl.append(len(labelling))   #positions of all the chunks in 
        labelling.append(perChunkLabel)
    
    return labelling, chunks, wav
    # return the text for all chunks detected



Loaded the voice encoder model on cpu in 0.11 seconds.


In [6]:
global_label, chunk, wav = get_large_audio_transcription(wav_fpath)




# with open('labellings/' + filename + '.json', 'w') as fp:
#     json.dump(labelling, fp)


start preprocessing:audio-chunks/chunk1.wav
preprocessing done:audio-chunks/chunk1.wav
create labels:audio-chunks/chunk1.wav
start preprocessing:audio-chunks/chunk2.wav
preprocessing done:audio-chunks/chunk2.wav
create labels:audio-chunks/chunk2.wav
start preprocessing:audio-chunks/chunk3.wav
preprocessing done:audio-chunks/chunk3.wav
create labels:audio-chunks/chunk3.wav


In [7]:
global_label

[[['1', 0, 37.4],
  ['0', 37.4, 40.22],
  ['1', 40.22, 40.76],
  ['0', 40.76, 42.44],
  ['1', 42.44, 43.34],
  ['0', 43.34, 48.5],
  ['1', 48.5, 49.16],
  ['0', 49.16, 60.56],
  ['1', 60.56, 96.62],
  ['0', 96.62, 96.68],
  ['1', 96.68, 96.74],
  ['0', 96.74, 98.54],
  ['1', 98.54, 99.98],
  ['0', 99.98, 104.0],
  ['1', 104.0, 110.18]],
 [['0', 0, 9.38],
  ['1', 9.38, 17.06],
  ['0', 17.06, 24.68],
  ['1', 24.68, 24.8],
  ['0', 24.8, 27.2],
  ['1', 27.2, 27.5],
  ['0', 27.5, 27.56],
  ['1', 27.56, 27.68],
  ['0', 27.68, 29.48],
  ['1', 29.48, 29.54],
  ['0', 29.54, 41.96],
  ['1', 41.96, 42.56],
  ['0', 42.56, 45.02],
  ['1', 45.02, 46.88],
  ['0', 46.88, 49.22],
  ['1', 49.22, 50.9],
  ['0', 50.9, 70.88],
  ['1', 70.88, 71.24],
  ['0', 71.24, 71.42],
  ['1', 71.42, 73.52],
  ['0', 73.52, 74.0],
  ['1', 74.0, 78.08],
  ['0', 78.08, 78.26],
  ['1', 78.26, 78.32],
  ['0', 78.32, 81.2],
  ['1', 81.2, 82.46],
  ['0', 82.46, 90.5],
  ['1', 90.5, 91.46],
  ['0', 91.46, 93.14],
  ['1', 93.14,

In [8]:
def changeBits(chunkLabel): 
    for i in range(len(chunkLabel)):
        if chunkLabel[i][0] =='0': 
            chunkLabel[i][0] = '1'
        else: 
            chunkLabel[i][0] = '0'
    return chunkLabel

In [9]:

def get_similarity(embed_i, embed_j):
    if (1-cosine(embed_i, embed_j)) >= 0.8 :
        return 1
    else:
        return 0
 
   

In [10]:
if global_label[0][-1][0] == '1':
    last_1_time = global_label[0][-1][1:]
else: 
    last_1_time = global_label[0][-2][1:]

embed1_time = wav[int(last_1_time[0]*sampling_rate):int(last_1_time[1]*sampling_rate)]


In [11]:
constant = global_label[0][-1][2]
constant

110.18

In [12]:
new_global = global_label.copy()


In [13]:
shouldChange = []
new_global.pop(0)
k=0
for i in new_global: 
    start_i, end_i= i[1][1], i[1][2]
    print(start_i)
    embedi_time = wav[int((start_i+global_label[k][-1][2])*1000):int((end_i+global_label[k][-1][2])*1000)]
    k += 1 
    
    if get_similarity(encoder.embed_utterance(embed1_time), encoder.embed_utterance(embedi_time)): 
        if i[-1][0] == '0': 
            # '0' -> '1'
            # '1' -> '0'
            changeBits(i)
            shouldChange.append(True)
        else: 
            shouldChange.append(False)
    else: 
        if i[-1][0] == '0': 
#             continue
            shouldChange.append(False)
        else: 
            shouldChange.append(True)
            # '0' -> '1'
            # '1' -> '0'
            changeBits(i)
            


9.38
16.7


In [14]:
embedi_time = wav[int((start_i+global_label[k][-1][2])*sampling_rate):int((end_i+global_label[k][-1][2])*sampling_rate)]
get_similarity(encoder.embed_utterance(embed1_time), encoder.embed_utterance(embedi_time))

1

In [15]:
new_global

[[['0', 0, 9.38],
  ['1', 9.38, 17.06],
  ['0', 17.06, 24.68],
  ['1', 24.68, 24.8],
  ['0', 24.8, 27.2],
  ['1', 27.2, 27.5],
  ['0', 27.5, 27.56],
  ['1', 27.56, 27.68],
  ['0', 27.68, 29.48],
  ['1', 29.48, 29.54],
  ['0', 29.54, 41.96],
  ['1', 41.96, 42.56],
  ['0', 42.56, 45.02],
  ['1', 45.02, 46.88],
  ['0', 46.88, 49.22],
  ['1', 49.22, 50.9],
  ['0', 50.9, 70.88],
  ['1', 70.88, 71.24],
  ['0', 71.24, 71.42],
  ['1', 71.42, 73.52],
  ['0', 73.52, 74.0],
  ['1', 74.0, 78.08],
  ['0', 78.08, 78.26],
  ['1', 78.26, 78.32],
  ['0', 78.32, 81.2],
  ['1', 81.2, 82.46],
  ['0', 82.46, 90.5],
  ['1', 90.5, 91.46],
  ['0', 91.46, 93.14],
  ['1', 93.14, 93.5],
  ['0', 93.5, 104.24]],
 [['0', 0, 16.7],
  ['1', 16.7, 16.88],
  ['0', 16.88, 19.1],
  ['1', 19.1, 23.3],
  ['0', 23.3, 25.88],
  ['1', 25.88, 27.86],
  ['0', 27.86, 33.44],
  ['1', 33.44, 33.62],
  ['0', 33.62, 34.1],
  ['1', 34.1, 39.56],
  ['0', 39.56, 45.5],
  ['1', 45.5, 45.86],
  ['0', 45.86, 46.1],
  ['1', 46.1, 46.58],
 

In [16]:
new_global.insert(0,global_label[0])
new_global

[[['1', 0, 37.4],
  ['0', 37.4, 40.22],
  ['1', 40.22, 40.76],
  ['0', 40.76, 42.44],
  ['1', 42.44, 43.34],
  ['0', 43.34, 48.5],
  ['1', 48.5, 49.16],
  ['0', 49.16, 60.56],
  ['1', 60.56, 96.62],
  ['0', 96.62, 96.68],
  ['1', 96.68, 96.74],
  ['0', 96.74, 98.54],
  ['1', 98.54, 99.98],
  ['0', 99.98, 104.0],
  ['1', 104.0, 110.18]],
 [['0', 0, 9.38],
  ['1', 9.38, 17.06],
  ['0', 17.06, 24.68],
  ['1', 24.68, 24.8],
  ['0', 24.8, 27.2],
  ['1', 27.2, 27.5],
  ['0', 27.5, 27.56],
  ['1', 27.56, 27.68],
  ['0', 27.68, 29.48],
  ['1', 29.48, 29.54],
  ['0', 29.54, 41.96],
  ['1', 41.96, 42.56],
  ['0', 42.56, 45.02],
  ['1', 45.02, 46.88],
  ['0', 46.88, 49.22],
  ['1', 49.22, 50.9],
  ['0', 50.9, 70.88],
  ['1', 70.88, 71.24],
  ['0', 71.24, 71.42],
  ['1', 71.42, 73.52],
  ['0', 73.52, 74.0],
  ['1', 74.0, 78.08],
  ['0', 78.08, 78.26],
  ['1', 78.26, 78.32],
  ['0', 78.32, 81.2],
  ['1', 81.2, 82.46],
  ['0', 82.46, 90.5],
  ['1', 90.5, 91.46],
  ['0', 91.46, 93.14],
  ['1', 93.14,

In [17]:
def joinlabels(orderedLabelling):
    combinedLabellings = []
    secondsCount= 0
    for i in range(noOfFiles):
        tempArray = orderedLabelling[i]
        isSameAsLastSpeaker=False
        
        if (len(combinedLabellings) == 0):
            combinedLabellings.extend(tempArray)
            
        else:
            if (combinedLabellings[len(combinedLabellings)-1][0] == tempArray[0][0]):
                combinedLabellings[len(combinedLabellings)-1][2] = combinedLabellings[len(combinedLabellings)-1][2]+ tempArray[0][2]
                isSameAsLastSpeaker=True
            for j in range(len(tempArray)):
                if (j > 0 or isSameAsLastSpeaker == False):
                    tempArray[j][1] = tempArray[j][1] + secondsCount # 135.5*(i)
                    tempArray[j][2] = tempArray[j][2] +secondsCount # 135.5*(i)
                    combinedLabellings.append(tempArray[j])
        print( combinedLabellings[len(combinedLabellings)-1][2])
        secondsCount = combinedLabellings[len(combinedLabellings)-1][2]
    return combinedLabellings

def createLabellings(filepath):
    for i in range(noOfFiles):
        with open(filepath +'/interview' + str(i) + '.json', 'r') as data_file:
            json_data = data_file.read()
        samplelabellings.append(json.loads(json_data))
noOfFiles = len(new_global)

In [18]:
combinedLabellings = joinlabels(new_global)
print(combinedLabellings)


110.18
214.42000000000002
269.94
[['1', 0, 37.4], ['0', 37.4, 40.22], ['1', 40.22, 40.76], ['0', 40.76, 42.44], ['1', 42.44, 43.34], ['0', 43.34, 48.5], ['1', 48.5, 49.16], ['0', 49.16, 60.56], ['1', 60.56, 96.62], ['0', 96.62, 96.68], ['1', 96.68, 96.74], ['0', 96.74, 98.54], ['1', 98.54, 99.98], ['0', 99.98, 104.0], ['1', 104.0, 110.18], ['0', 110.18, 119.56], ['1', 119.56, 127.24000000000001], ['0', 127.24000000000001, 134.86], ['1', 134.86, 134.98000000000002], ['0', 134.98000000000002, 137.38], ['1', 137.38, 137.68], ['0', 137.68, 137.74], ['1', 137.74, 137.86], ['0', 137.86, 139.66], ['1', 139.66, 139.72], ['0', 139.72, 152.14000000000001], ['1', 152.14000000000001, 152.74], ['0', 152.74, 155.20000000000002], ['1', 155.20000000000002, 157.06], ['0', 157.06, 159.4], ['1', 159.4, 161.08], ['0', 161.08, 181.06], ['1', 181.06, 181.42000000000002], ['0', 181.42000000000002, 181.60000000000002], ['1', 181.60000000000002, 183.7], ['0', 183.7, 184.18], ['1', 184.18, 188.26], ['0', 188.26

In [19]:
    for i in combinedLabellings:
        if(i[2]-i[1]<1):
            combinedLabellings.remove(i)

In [20]:
combinedLabellings

[['1', 0, 37.4],
 ['0', 37.4, 40.22],
 ['0', 40.76, 42.44],
 ['0', 43.34, 48.5],
 ['0', 49.16, 60.56],
 ['1', 60.56, 96.62],
 ['1', 96.68, 96.74],
 ['0', 96.74, 98.54],
 ['1', 98.54, 99.98],
 ['0', 99.98, 104.0],
 ['1', 104.0, 110.18],
 ['0', 110.18, 119.56],
 ['1', 119.56, 127.24000000000001],
 ['0', 127.24000000000001, 134.86],
 ['0', 134.98000000000002, 137.38],
 ['0', 137.68, 137.74],
 ['0', 137.86, 139.66],
 ['0', 139.72, 152.14000000000001],
 ['0', 152.74, 155.20000000000002],
 ['1', 155.20000000000002, 157.06],
 ['0', 157.06, 159.4],
 ['1', 159.4, 161.08],
 ['0', 161.08, 181.06],
 ['0', 181.42000000000002, 181.60000000000002],
 ['1', 181.60000000000002, 183.7],
 ['1', 184.18, 188.26],
 ['1', 188.44, 188.5],
 ['0', 188.5, 191.38],
 ['1', 191.38, 192.64],
 ['0', 192.64, 200.68],
 ['0', 201.64, 203.32],
 ['0', 203.68, 231.12],
 ['0', 231.3, 233.52],
 ['1', 233.52, 237.72000000000003],
 ['0', 237.72000000000003, 240.3],
 ['1', 240.3, 242.28000000000003],
 ['0', 242.28000000000003, 2

In [6]:
#!/usr/bin/env python3

from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import srt
import json
import datetime
import sys


wav_fpath = "file.wav"
SetLogLevel(-1)

if not os.path.exists("model"):
    print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
    exit (1)

sample_rate=16000
model = Model("/home/milindsoni/Documents/wisepal_all_features/wisepal/Resemblyzer-master/python/example/model")
rec = KaldiRecognizer(model, sample_rate,"flipkart")
rec.SetWords(True)
# sys.argv[1],
process = subprocess.Popen(['ffmpeg', '-loglevel', 'quiet', '-i', wav_fpath ,
                            '-ar', str(sample_rate) , '-ac', '1', '-f', 's16le', '-'],
                            stdout=subprocess.PIPE)
#rec = KaldiRecognizer(model, 16000, "zero oh one two three four five six seven eight nine")

WORDS_PER_LINE = 7

def transcribe():
    results = []
    subs = []
    while True:
       data = process.stdout.read(4000)
       if len(data) == 0:
           break
       if rec.AcceptWaveform(data):
           results.append(rec.Result())
    results.append(rec.FinalResult())

    for i, res in enumerate(results):
       jres = json.loads(res)
       if not 'result' in jres:
           continue
       words = jres['result']
       for j in range(0, len(words), WORDS_PER_LINE):
           line = words[j : j + WORDS_PER_LINE] 
           s = srt.Subtitle(index=len(subs), 
                   content=" ".join([l['word'] for l in line]),
                   start=datetime.timedelta(seconds=line[0]['start']), 
                   end=datetime.timedelta(seconds=line[-1]['end']))
           subs.append(s)
    return subs

# 
with open(wav_fpath.split('.')[0]+ '.txt', 'w') as f:
    f.write((srt.compose(transcribe())))
    



TypeError: string indices must be integers

In [12]:
# print(srt.compose(transcribe()))
# hellow? 
# transcrib
subs.transcribe()

NameError: name 'subs' is not defined

In [1]:
subs

NameError: name 'subs' is not defined

In [22]:
combinedLabellings

[['1', 0, 37.4],
 ['0', 37.4, 40.22],
 ['0', 40.76, 42.44],
 ['0', 43.34, 48.5],
 ['0', 49.16, 60.56],
 ['1', 60.56, 96.62],
 ['1', 96.68, 96.74],
 ['0', 96.74, 98.54],
 ['1', 98.54, 99.98],
 ['0', 99.98, 104.0],
 ['1', 104.0, 110.18],
 ['0', 110.18, 119.56],
 ['1', 119.56, 127.24000000000001],
 ['0', 127.24000000000001, 134.86],
 ['0', 134.98000000000002, 137.38],
 ['0', 137.68, 137.74],
 ['0', 137.86, 139.66],
 ['0', 139.72, 152.14000000000001],
 ['0', 152.74, 155.20000000000002],
 ['1', 155.20000000000002, 157.06],
 ['0', 157.06, 159.4],
 ['1', 159.4, 161.08],
 ['0', 161.08, 181.06],
 ['0', 181.42000000000002, 181.60000000000002],
 ['1', 181.60000000000002, 183.7],
 ['1', 184.18, 188.26],
 ['1', 188.44, 188.5],
 ['0', 188.5, 191.38],
 ['1', 191.38, 192.64],
 ['0', 192.64, 200.68],
 ['0', 201.64, 203.32],
 ['0', 203.68, 231.12],
 ['0', 231.3, 233.52],
 ['1', 233.52, 237.72000000000003],
 ['0', 237.72000000000003, 240.3],
 ['1', 240.3, 242.28000000000003],
 ['0', 242.28000000000003, 2

In [23]:
# #!/usr/bin/env python3

# from vosk import Model, KaldiRecognizer
# import sys
# import json
# import os

# if not os.path.exists("model"):
#     print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
#     exit (1)


# model = Model("/home/milindsoni/Documents/wisepal_all_features/wisepal/Resemblyzer-master/python/example/model")

# # Large vocabulary free form recognition
# rec = KaldiRecognizer(model, 16000)

# # You can also specify the possible word list
# #rec = KaldiRecognizer(model, 16000, "zero oh one two three four five six seven eight nine")
# directory = "temp"
# for filename in os.listdir(directory):
#     wf = open(os.path.join(directory,filename), "rb")
#     wf.read(44) # skip header

#     while True:
#         data = wf.read(4000)
#         if len(data) == 0:
#             break
#         if rec.AcceptWaveform(data):
#             res = json.loads(rec.Result())
#             print (res['text'])

#     res = json.loads(rec.FinalResult())
#     print (res['text'])

In [24]:
mappings = combinedLabellings
mappings

[['1', 0, 37.4],
 ['0', 37.4, 40.22],
 ['0', 40.76, 42.44],
 ['0', 43.34, 48.5],
 ['0', 49.16, 60.56],
 ['1', 60.56, 96.62],
 ['1', 96.68, 96.74],
 ['0', 96.74, 98.54],
 ['1', 98.54, 99.98],
 ['0', 99.98, 104.0],
 ['1', 104.0, 110.18],
 ['0', 110.18, 119.56],
 ['1', 119.56, 127.24000000000001],
 ['0', 127.24000000000001, 134.86],
 ['0', 134.98000000000002, 137.38],
 ['0', 137.68, 137.74],
 ['0', 137.86, 139.66],
 ['0', 139.72, 152.14000000000001],
 ['0', 152.74, 155.20000000000002],
 ['1', 155.20000000000002, 157.06],
 ['0', 157.06, 159.4],
 ['1', 159.4, 161.08],
 ['0', 161.08, 181.06],
 ['0', 181.42000000000002, 181.60000000000002],
 ['1', 181.60000000000002, 183.7],
 ['1', 184.18, 188.26],
 ['1', 188.44, 188.5],
 ['0', 188.5, 191.38],
 ['1', 191.38, 192.64],
 ['0', 192.64, 200.68],
 ['0', 201.64, 203.32],
 ['0', 203.68, 231.12],
 ['0', 231.3, 233.52],
 ['1', 233.52, 237.72000000000003],
 ['0', 237.72000000000003, 240.3],
 ['1', 240.3, 242.28000000000003],
 ['0', 242.28000000000003, 2

In [25]:
mappings = combinedLabellings


print(len(mappings))

def clubbing(mappings):
    clubbed = []
    starIndex = mappings[0][0]
    i = 0

    while i<len(mappings):
        temp = mappings[i] 
        k = i+1
        while k<len(mappings) and mappings[k][0]==starIndex: 
            k+=1

        i = k

        if k>=len(mappings): # explicit ending condition
            k = len(mappings)-1
            i = len(mappings)
            temp[-1] = mappings[-1][-1]
            clubbed.append(temp)
            break

        starIndex = mappings[k][0]
        temp[-1] = mappings[k-1][-1]
        clubbed.append(temp)
    return clubbed
clubbed = clubbing(mappings)

for i in clubbed: 
    print(i)



44
['1', 0, 37.4]
['0', 37.4, 60.56]
['1', 60.56, 96.74]
['0', 96.74, 98.54]
['1', 98.54, 99.98]
['0', 99.98, 104.0]
['1', 104.0, 110.18]
['0', 110.18, 119.56]
['1', 119.56, 127.24000000000001]
['0', 127.24000000000001, 155.20000000000002]
['1', 155.20000000000002, 157.06]
['0', 157.06, 159.4]
['1', 159.4, 161.08]
['0', 161.08, 181.60000000000002]
['1', 181.60000000000002, 188.5]
['0', 188.5, 191.38]
['1', 191.38, 192.64]
['0', 192.64, 233.52]
['1', 233.52, 237.72000000000003]
['0', 237.72000000000003, 240.3]
['1', 240.3, 242.28000000000003]
['0', 242.28000000000003, 248.52]
['1', 248.52, 253.98000000000002]
['0', 253.98000000000002, 264.84000000000003]
['1', 264.84000000000003, 265.98]
['0', 265.98, 269.22]


In [26]:
clubbed = [['1', 0, 37.4],
['0', 37.4, 60.56],
['1', 60.56, 96.74],
['0', 96.74, 98.54],
['1', 98.54, 99.98],
['0', 99.98, 104.0],
['1', 104.0, 110.18],
['0', 110.18, 119.56],
['1', 119.56, 127.24000000000001],
['0', 127.24000000000001, 155.20000000000002],
['1', 155.20000000000002, 157.06],
['0', 157.06, 159.4],
['1', 159.4, 161.08],
['0', 161.08, 181.60000000000002],
['1', 181.60000000000002, 188.5],
['0', 188.5, 191.38],
['1', 191.38, 192.64],
['0', 192.64, 233.52],
['1', 233.52, 237.72000000000003],
['0', 237.72000000000003, 240.3],
['1', 240.3, 242.28000000000003],
['0', 242.28000000000003, 248.52],
['1', 248.52, 253.98000000000002],
['0', 253.98000000000002, 264.84000000000003],
['1', 264.84000000000003, 265.98],
['0', 265.98, 269.22]]

In [28]:
# filename.split('.')[0] +
file = open( 'file.txt', 'r')
text = file.readlines()
sets = []
temp = []
for i in range(len(text)): 
    
    if (i+1)%4==0: 
        sets.append(temp)
        temp = []
        continue
    temp.append(text[i])

# filter 1: to remove \n 
for i in range(len(sets)):
    s = sets[i]
    for j in range(3): 
        sets[i][j] = sets[i][j].split('\n')[0]
# filter 2: to remove --> from time periods!
for i in range(len(sets)): 
    sets[i][1] =  sets[i][1].split(' --> ')

def convertString2Time(string): 
    
    s = string.replace(',', '.')
    s = s.split(":")
    # print(s,end = ' ')
    mult = (60)**(len(s)-1)
    for i in range(len(s)): 
        s[i] = float(s[i])*mult
        mult/=60
    # print(s)
    return sum(s)
# To expand each array!
for i in range(len(sets)): 
    temp = []
    for j in range(2):
        temp.append(convertString2Time(sets[i][1][j]))
    sets[i][1] = temp

s = [] 
for i in sets:
    temp = [i[0], i[1][0], i[1][1], i[2]]
    s.append(temp)
sets = s

import pandas as pd
from IPython.display import display
df = pd.DataFrame(sets, columns = ['index', 'start', 'end', 'sentence'])
df = df.set_index('index')
display(df)
df.to_csv('new_transcript.csv')

,start,end,sentence
index,,,
1,0.021,1.681,threats and i want
2,2.916,5.461,what he a squad car and when
3,5.461,8.040,everything that was the coin about fifteen
4,8.040,11.070,percent a cult additions to the traditions
5,11.073,12.577,were shopping her in the last three
...,...,...,...
101,289.978,291.600,because we are closing it down on
102,291.608,292.800,the mobile app so we'll see
103,292.800,294.186,yeah all the steps of the user


In [29]:
import pandas as pd
df = pd.read_csv('new_transcript.csv')
k = 0
npdf = df.values
labels = []
for i in range(len(npdf)): 
    if k>=len(clubbed): 
        labels.append(clubbed[-1][0])
        continue
        
    if npdf[i][2]<clubbed[k][2]:
        labels.append(clubbed[k][0])
    elif k<len(clubbed): 
        labels.append(clubbed[k][0])
        k+=1
df['labels'] = labels
df.to_csv('withLabels.csv', index = False)
df = pd.DataFrame(clubbed, columns = ['label', 'star', 'end'])
df.to_csv('clubbed.csv', index = False)


In [30]:
transcript = pd.read_csv("withlabels.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'withlabels.csv'